In [1]:
# !pip install numba & conda install cudatoolkit

In [2]:
# !pip install ta
# !pip install pandas-ta

In [1]:
################# imports #############

from utils import *
print(NqPath)

C:/Users/kasra/Downloads/Ten-Surf/Tensurf-RL/tensurfrl/data/LOB_NQU22-CME_2_1_10_20days.lob


In [2]:
##### gpu supporting################################

#from numba import jit, cuda





################ data preprocessing  for forex data ###########
# preProcessedData = dataAnalyse(data, sequenceNumber, testNumbers)
# trainData, testData = preProcessedData.dataPreProcessing()

# differenceBetweenNormalDataAndActualData = len(data_) - (len(trainData) + len(testData))

# dataTrain = data_.iloc[ differenceBetweenNormalDataAndActualData:-int(len(data_) * 0.1) , : ]
# dataTrain.reset_index(drop=True, inplace=True)
# dataTest = data_.iloc[-int(len(data_) * 0.1) : , : ]
# dataTest.reset_index(drop=True, inplace=True) 

################ data preprocessing  for forex STOCK ###########
preProcessedData = lobPreprocessing(NqPath)
trainData, testData, dataTrain, dataTest = preProcessedData.final()

###################################### loop fr train ###########

      ################## initialize ##################
       
modelSelection = modelForQ(trainData, outPutOfQ)
Q = modelSelection.fullyConnectedModel()
env = dict()
memory = deque(maxlen = relayMemorySize)
selectedAction = actions[random.randint(0, len(actions)-1)]

      #################################################

#len(trainData)-1

#!@jit(target ="cuda")

for i in range(len(trainData)-1):

    ############################ current state #############
    state = trainData[i, :]
    
    ########################### transition between state and action #####################
    env_ = openOrClosePosition(env, selectedAction, i, dataTrain, margin,  Sl, Tp, levrage)   
    env = env_.tradeCheck() 

    ################################## next observation #################                  
    nextState = trainData[i+1, :] 
    
    ################################# reard calculation ######
    rewardCalculating = rewardCalculation(env, i, selectedAction, dataTrain, stepSwith = True)
    reward = rewardCalculating.reward()
    
    ################################### saving to memory ###################
    memory.append([state, actionConverting[actions.index(selectedAction)], reward, nextState])       
    
    
    ############################## action selection ####
    
    if len(memory) >= numberOfFirstBatchData:
        
        trainer = updatinQtable(memory, batchSize, Q, gamma, nextState)
        Q = trainer.updating()
        print("Q was updated")
        if epsilon > epsilonFinal:
          epsilon *= epsilonDecay

        memory = deque(maxlen = relayMemorySize)
    
    ###########################################################  select action according to decided policy ##################

    policy_ = policy(Q, actions, state, epsilon)
    selectedAction = policy_.actionSelection()
    
    
    ############################# state updating ##########################
    state = nextState
    
    ############################## save Q ##########################


Q.save("DQN.csv")
    
    


############################## test section #########################

envTest = dict()
memoryTest = deque(maxlen = relayMemorySize)
selectedAction = actions[random.randint(0,len(actions)-1)]

# len(testData)-1

for i in range(len(testData)-1):
    
    
    
    ############################ current state #############
    state = trainData[i, :]

    ########################### transition between state and action #####################
    env_ = openOrClosePosition(envTest, selectedAction, i, dataTest, margin,  Sl, Tp, levrage)   
    envTest = env_.tradeCheck() 

    ################################## next observation #################                  
    nextState = trainData[i+1, :] 
    
    ################################# reard calculation ######
    rewardCalculating = rewardCalculation(envTest, i, selectedAction, dataTest, stepSwith = True)
    reward = rewardCalculating.reward()
    
    ################################### saving to memory ###################
    memoryTest.append([state, actionConverting[actions.index(selectedAction)], reward, nextState])       
    
    
    ############################## action selection ####
    
    if len(memoryTest) >= numberOfFirstBatchData:
        
        trainer = updatinQtable(memoryTest, batchSize, Q, gamma, nextState)
        Q = trainer.updating()
        print("Q test was updated")
        if epsilonTest > epsilonFinal:
          epsilonTest *= epsilonDecay
    
    ###########################################################  select action according to decided policy ##################

    policy_ = policy(Q, actions, state, epsilonTest)
    selectedAction = policy_.actionSelection()
    
    
    ############################# state updating ##########################
    state = nextState


Q.save("DQNTest.csv")


################## show results #############################
showTraderResults_ = showTraderResults(envTest, dataTest["close"].values)
statements = showTraderResults_.final()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                984       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 1,659
Trainable params: 1,659
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 28ms/step
Q was updated
1/1 [==============================] - 0s 26ms/step
Q was updated
1/1 [==============================] - 0s 26ms/step
Q was updated
1/1 [==============================] - 0s 19ms/step
Q was updated
1/1 [==============================] - 0s 20ms/step
Q 

KeyboardInterrupt: 